In [2]:
!python --version

Python 2.7.18


In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import pickle
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./data/test.tsv
./data/dev.tsv
./data/train.tsv


In [5]:
## loading datasets
train_df = pd.read_csv('./data/train.tsv', sep='\t')
dev_df = pd.read_csv('./data/dev.tsv', sep='\t')
test_df = pd.read_csv('./data/test.tsv', sep='\t')

train_df.shape, dev_df.shape, test_df.shape

((1448739, 2), (160993, 2), (1557858, 2))

In [6]:
## inspect train set
train_df.head()

,query,target
0,How is the life of a math student? Could you d...,1
1,life math student describe own experiences,0
2,Which level of prepration is enough for the ex...,1
3,level enough jlpt5,0
4,How do I control my horny emotions,1


In [7]:
## sampling only first 10k train samples to bootstrap a quick test.
train_text = train_df['query'].values[:10000]
test_text = test_df['query'].values

## TF-IDF based features
vectorizer = TfidfVectorizer(ngram_range=(1,3), 
                             min_df=0.001, 
                             max_df=0.7, 
                             analyzer='word')

X_train = vectorizer.fit_transform(train_text)
X_test = vectorizer.transform(test_text)


y_train = train_df.target.values[:10000]
y_test = test_df.target.values
## TODO:
## can be improved with cross validation to improve generalization
gb = GradientBoostingClassifier(n_estimators = 400, random_state=0)


gb.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=400, random_state=0)

In [8]:
predictions_rf = gb.predict(X_test)

## f1 score of 99% comparative to attention based model
print(classification_report(y_test, predictions_rf))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    775929
           1       1.00      0.97      0.99    781929

    accuracy                           0.99   1557858
   macro avg       0.99      0.99      0.99   1557858
weighted avg       0.99      0.99      0.99   1557858



In [9]:


with open('query_classifier.pickle', 'wb') as handle:
    pickle.dump(gb, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [10]:

with open('query_vectorizer.pickle', 'wb') as handle:
    pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Testing SPAADIA/SQUAD BASED CLASSIFIER ON QUORA DATASET

In [11]:
import urllib
query_vectorizer_spaadia_squad = pickle.load(
            urllib.request.urlopen(
                "https://raw.githubusercontent.com/shahrukhx01/ocr-test/main/query_vectorizer.pickle"))
query_classifier_spaadia_squad = pickle.load(
            urllib.request.urlopen(
                "https://raw.githubusercontent.com/shahrukhx01/ocr-test/main/query_classifier.pickle"))

/tmp/ipykernel_35845/3408061569.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  query_vectorizer_spaadia_squad = pickle.load(
/home/lb/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/lb/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/h

In [35]:
vectorizer_for_text = pickle.load(open("query_vectorizer.pickle", 'rb'))
classifier_for_text = pickle.load(open("query_classifier.pickle",'rb'))

In [72]:
test_text = test_df['query'].values
X_test = vectorizer_for_text.transform(test_text)
predictions_rf = classifier_for_text.predict(X_test)

## f1 score of 99% comparative to attention based model
print(classification_report(y_test, predictions_rf))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    775929
           1       1.00      0.97      0.99    781929

    accuracy                           0.99   1557858
   macro avg       0.99      0.99      0.99   1557858
weighted avg       0.99      0.99      0.99   1557858



In [32]:
ls  = ["can we do this?"]
text = query_vectorizer_spaadia_squad.transform(ls)
query_classifier_spaadia_squad.predict(text)

array([0])

In [73]:
ls  = ["can we do this?"]
text = vectorizer_for_text.transform(ls)
classifier_for_text.predict(text)

array([1])